# GeoTIFFs to reV HDF5 Files

In the previous tutorial, we demostrated how we can use `reVX`'s Geotiff handler to manage geotiff files

In this tutorial, we will go over getting tiff files into a [reV](https://github.com/NREL/reV)-ready format using the `LayeredH5` handler

Let's start with a few common imports:

In [1]:
import urllib.request
from pathlib import Path

from reVX.handlers.layered_h5 import LayeredH5
from reVX.handlers.geotiff import Geotiff

## Downloading the data

Before we dive into the code, we first have to download a sample TIFF from 
[Siting Lab](https://data.openei.org/submissions/6119) to use as an example of adding data to a layered HDF5 file. 
In particular, we will be using data from {cite:t}`oedi_6121`.

If you have already downloaded the data, you can skip this step (just make sure path variables below are set correctly).
We'll start by defining the local file path destination:

In [2]:
TEMPLATE_TIFF = "NEXRAD_green_los.tif"

Next, we can write a short download function using `urllib`,
which is part of the Python standard library, and call it to download the data.

In [3]:
NEXRAD_TIFF_URL = "https://data.openei.org/files/6121/nexrad_4km.tif"

def download_nexrad_tiff(local_filepath):
    if Path(local_filepath).exists():
        print(f"{str(local_filepath)!r} already exists!")
        return

    urllib.request.urlretrieve(NEXRAD_TIFF_URL, local_filepath)
    print(f"Downloaded {str(local_filepath)!r}!")

download_nexrad_tiff(TEMPLATE_TIFF)

'NEXRAD_green_los.tif' already exists!


## Creating the layered HDF5 from TIFF

First, we will initialize the `LayeredH5` object.

If creating a new HDF5 file that does not exist, we use the `.create_new()` method. 

When creating a new HDF5, a template filepath must be specified. The template file is used to define the properties of the HDF5 file including:
1. The profile information
2. Coordinate reference system and projection
3. The geographic extent, spatial resolution

All other files that are subsequently added to the HDF5 file will be transformed/adjusted to fit the properties of the template file before being written to the file.

In [4]:
H5_PATH = "example.h5"

# Initialize layered h5 object
h5 = LayeredH5(H5_PATH, template_file=TEMPLATE_TIFF)

# If file doesn't exist, create new h5
h5.create_new()

: 

Inspecting the h5 file (using the `layers` property), we see that the first two layers are longitude and latitude arrays. These are the coordinate locations for each grid cell defined by the template file pixels.

In [4]:
# Use the layer method to see the layers in the H5 file
h5.layers

NameError: name 'h5' is not defined

Meta data information about the h5 file can be retrieved by using `.profile` and `.shape`

In [21]:
print(f"H5 profile: {h5.profile}")
print(f"shape: {h5.shape}")

H5 profile: {'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 255.0, 'width': 52005, 'height': 32697, 'count': 1, 'crs': '+init=epsg:5070', 'transform': (90.0, 0.0, -2400019.7367069316, 0.0, -90.0, 3197068.2309463923), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzma', 'interleave': 'band'}
shape: (32697, 52005)


Once the H5 file is created(or if it exists already), we can write numpy arrays and tiff files into the h5 files using the `.write_layer_to_h5()` and `.write_geotiff_to_h5()` respectively

In [10]:
# adding numpy arrays

#Let's read in a geotiff file into a numpy array using the Geotiff handler
filepath = DATA.joinpath("nexrad_green_los.tif")
with Geotiff(filepath) as geo:
    arr = geo.values #np array
    profile = geo.profile

# add to h5
h5.write_layer_to_h5(values=arr,
                     layer_name=filepath.stem,
                     profile=profile,
                     description='NEXRAD Line of sight')

In [11]:
#adding a geotiff file directly
filepath = DATA.joinpath("airport_heliport_setbacks.tif")
h5.write_geotiff_to_h5(geotiff=filepath,
                       layer_name=filepath.stem,
                       description='Setbacks from airports and heliports',
                       replace=False)


Now we can check to see what layers are currently in the H5 file

In [24]:
# Checking current layers in the
h5.layers

['airport_heliport_setbacks', 'latitude', 'longitude', 'nexrad_green_los']

We can also add multiple geotiffs into the h5 using the `.layers_to_h5()` method. 

This accepts a list or dictionary mapping layer name to geotiff filepaths. You can also pass a dictionary mapping layer name to description for the `description` argument

In [12]:
file_list = list(DATA.glob("*.tif"))[:3]

print(f"Adding {len(file_list)} file(s) to the h5...")
for file in file_list:
    print(file.stem)

h5.layers_to_h5(
    layers=file_list,
    replace=False
)

Adding 3 file(s) to the h5...
setbacks_pipeline_reference
setbacks_structure_reference
setbacks_structure_115hh_170rd


In [13]:
# Checking current layers in the h5
h5.layers

['airport_heliport_setbacks',
 'latitude',
 'longitude',
 'nexrad_green_los',
 'setbacks_pipeline_reference',
 'setbacks_structure_115hh_170rd',
 'setbacks_structure_reference']

Layers in the h5 file can also be extracted as geotiffs. `.layer_to_geotiff()` for single layers, `.extract_layers()` for multiple layers.

All the layers in the h5 can be extracted using `.extract_all_layers()` by passing an output directory as argument.

In [28]:
#extracting single layer
layer = 'ops_water'
outpath = DATA.joinpath("ops_water_h5_extract.tif")
h5.layer_to_geotiff(layer=layer,
                    geotiff=outpath)


#Extracting multiple layers
layers = {
    'nexrad_green_los': DATA.joinpath("nexrad_green_los_h5_extract.tif"),
    'setbacks_pipeline_reference': DATA.joinpath("setbacks_pipeline_reference_h5_extract.tif")
}
h5.extract_layers(layers)

### Using the command line to add and extract layers from h5 files

Alternatively, the command line can be used to add and extract layers from the h5 file

**1. Adding tiffs to the h5** 

First, we need to construct a json config file that contains layer name mapping to geotiff filepaths
This json configuration file can optionally contain layer descriptions


`layers.json`
```json
{
    "layers":
        {
            "nexrad_green_los": "../data/nexrad_green_los.tif",
            "ops_water": "../data/ops_water.tif",
            "setbacks_transmission_reference": "../data/setbacks_transmission_reference.tif"
        }
}
```

Then we run `$ reVX exclusions layers-to-h5  -h5 "../data/example.h5" --layers layers.json` on the command line.

**2. Extracting layers from h5**

To extract layers from the h5 file, we pass the list of layers to extract as an argument and an output directory

Example: `$ reVX exclusions layers-from-h5 -h5 "../data/example.h5" -l nexrad_green_los ops_water setbacks_transmission_reference -o "./outputs"`